In [1]:
from IPython.core.display import display, HTML


In [2]:
display(HTML("<h8>Hello!</h8>"))

In [13]:
from bokeh.palettes import Category20, Category10
from goodreads10k.categories import get_categories

df_combine = books = pd.read_csv('./EmbeddingGraph/books_final_df.csv')
category_tags = [
    "Children-s",
    'Classics',
    'Fantasy',
    'Horror',
    'Memoir',
    'Mystery',
    'Paranormal',
    'Poetry',
    'Romance',
    'Thriller'
    ]
book_categories = get_categories(category_tags) 

book_category_series = books['goodreads_book_id'].apply(lambda bid: book_categories[bid] if bid in book_categories else 'other')
category_color = { ct.lower(): Category10[10][idx]  for idx, ct in enumerate(category_tags)}
df_combine['node_color'] = pd.Series([category_color[i] if i in category_color else Category20[20][15] for i in book_category_series ])

In [14]:
from bokeh.plotting import figure, show, output_notebook, save, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b
import pandas as pd
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput, RangeSlider, CheckboxGroup
import numpy as np 
from bokeh.layouts import column, layout
from bokeh.io import curdoc
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

output_notebook()

source = ColumnDataSource(dict(
    x=df_combine['x_tsne'],
    y=df_combine['y_tsne'],
    title= df_combine['title'],
    fill_color=df_combine['node_color'],
  #  legend_label=book_category_series
))

title = 'PCA visualization of embeddings'

plot_lda = figure(plot_width=2000, plot_height=600,
                     title='Find your boook!', tools="pan,wheel_zoom,box_zoom,reset,hover",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y',source=source,
                alpha=0.99, size=10,
                fill_color='fill_color') 
                #fill_color=factor_cmap('legend', palette=Category20b[20], factors=df_combine.category.unique()),
                #legend_group='legend_label')

hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title"}

background_color = 'white'

genre = CheckboxGroup(labels=category_tags, active=list(range(0, len(category_tags))), background=background_color, height=250, margin=(50,5,5,5))

year = RangeSlider(start=1600, end=2020, value=(1600,2020), step=1, title="Year of publication",  background=background_color, width=280)

average_rating =  RangeSlider(start=0, end=5, value=(0,5), step=.1, title="Rating",  background=background_color, width=280)

ratings_count = RangeSlider(start=0, end=1000000, value=(1,1000000), step=1, title="Amount of ratings",  background=background_color, width=280)

authors =  TextInput(value="", title="Author",  background=background_color, width=280)

title  =  TextInput(value="", title="Title",  background=background_color, width=280)


def select_books():
    
    selected =  df_combine.copy()
    selected = selected.loc[(selected["node_color"].isin( [ list(category_color.values())[x] for x in genre.active]))]
    
    selected = selected.loc[ (selected["original_publication_year"] <= year.value[1]) & (selected["original_publication_year"] >=year.value[0]) &
                 (selected["average_rating"] <= average_rating.value[1]) & (selected["average_rating"] >= average_rating.value[0] ) &
                 (selected["ratings_count"] <= ratings_count.value[1]) & (selected["ratings_count"] >= ratings_count.value[0] ) ]
     
    selected = selected.loc[selected["authors"].str.contains(authors.value) == True ] 
    selected = selected.loc[selected["title"].str.contains(title.value) == True] 

    return selected

def update():
    df = select_books()
    source.data = dict(
        x=df['x_tsne'],
        y=df['y_tsne'],
        title= df['title'],
        fill_color=df['node_color']
    )

    
controls = [authors, title, year, average_rating, ratings_count]    
for control in controls:
    control.on_change('value', lambda attr, old, new: update())
genre.on_change('active', lambda attr, old, new: update())
controls = [genre] + controls

inputs = column(*controls, width=300, height=200,  background=background_color, sizing_mode="fixed")

plot_lda.sizing_mode = "stretch_width"
l = layout([[inputs, plot_lda]]  , sizing_mode="stretch_both") 


update()  # initial load of the data


def modify_doc(doc):
    doc.add_root(l)


handler = FunctionHandler(modify_doc)
app = Application(handler)

show(app)

Loading BokehJS ...

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8889';                       use --allow-websocket-origin=localhost:8889 or set BOKEH_ALLOW_WS_ORIGIN=localhost:8889 to permit this; currently we allow origins {'localhost:8888'}


In [ ]:
from IPython.display import Image
Image(filename='legend.png') 

In [ ]:
display(HTML("<div></div>"))

In [ ]:
from ipywidgets import interact, fixed, FloatSlider, interact_manual, VBox, HBox, interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import data_preprocessing as proc
import visualisations as vis
from igraph import * 
import warnings
warnings.filterwarnings('ignore')


tag_limit = 100
how_many = 10
%store -r ids
%store -r matrix_authors
%store -r matrix_years
%store -r matrix_pages
%store -r matrix_rates
%store -r matrix_popularity
%store -r authors
%store -r titles

text = widgets.Text( description='Books list:')
display(text)


select = widgets.Dropdown(
    options=['None', 'List_1', 'List_2','List_3'],
    value='None',
    description='Selected list:',
)
display(select)

auth = FloatSlider(min=0, max=10, value=1, description="Author: ")
year = FloatSlider(min=0, max=10, value=1, description="Publication year: ")
pages = FloatSlider(min=0, max=10, value=0.4, description="Number of pages: ")
rates = FloatSlider(min=0, max=10, value=2, description="Author: ")
popularity = FloatSlider(min=0, max=10, value=0, description="Popularity: ")
shelves = FloatSlider(min=0, max=10, value=4, description="Subject: ")


#display(auth, year, pages, rates, popularity, shelves)

items = HBox([auth, year, pages, rates, popularity, shelves])

tlist3 = ['Rose Madder','The Eyes of the Dragon', 'Eleven Minutes', 'Twelfth Night',
'The Old Man and the Sea','Misery','A Briefer History of Time','The Universe in a Nutshell',
 "It's Not about the Bike: My Journey Back to Life",
 "The Universe in a Single Atom: The Convergence of Science and Spirituality",
 "In the Name of Jesus: Reflections on Christian Leadership","The Innocents Abroad",
 "To Kill a Mockingbird",
 "Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry",
 "Trump: The Art of the Deal", "Dracula",
 "Preface to the Presidency: Selected Speeches of Bill Clinton 1974-1992",
 "Lysis/Phaedrus/Symposium: Plato on Homosexuality", "Poems New and Collected"]

tlist1 = ['Rose Madder','The Eyes of the Dragon','The Old Man and the Sea','Misery',"To Kill a Mockingbird",
              "Trump: The Art of the Deal", "Dracula","Preface to the Presidency: Selected Speeches of Bill Clinton 1974-1992",]

tlist2 = ['A Briefer History of Time','The Universe in a Nutshell', 
               "In the Name of Jesus: Reflections on Christian Leadership",
              "Poems New and Collected", 'Twelfth Night',
              "Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry",
              "It's Not about the Bike: My Journey Back to Life", 
              "The Universe in a Single Atom: The Convergence of Science and Spirituality", "Misery"]

scal = {"auth":1, "years":1, "pages":1, "rates":1, "popularity":1, "shelves":1}

@interact_manual(auth=auth, year=year, pages=pages, rates=rates, popularity=popularity, shelves=shelves)
def update_graph(auth, year, pages, rates, popularity, shelves):
    scal = {"auth":auth, "years":year, "pages":pages, "rates":rates, "popularity":popularity, "shelves":shelves}
   
    if select.value == "None" : 
        if "," not in (text.value): 
            return "Give me at least two books"

        tlist = (text.value).split(",")
    else : 
        if select.value == "List_1" :
            tlist=tlist1
        if select.value == "List_2" : 
            tlist=tlist2
        if select.value == "List_3" : 
            tlist=tlist3
    g, visual_style = vis.visualuse_similarity("circular", how_many, tag_limit, ids, scal, titles, authors,
                         matrix_authors, matrix_years, matrix_pages,
                         matrix_rates, matrix_popularity, title_list=tlist)
    p = plot(g, **visual_style)
    p.save('graph.png')
    return Image(filename='graph.png')  
    




In [4]:
display(HTML("<div></div>"))

In [5]:
display(HTML("<div></div>"))

In [6]:
display(HTML("<div>Compare your books!</div>"))